This problem was asked by Snapchat.

Given an array of time intervals (start, end) for classroom lectures (possibly overlapping), find the minimum number of rooms required.

For example, given [(30, 75), (0, 50), (60, 150)], you should return 2.

Because no two lectures can simultaneously take place in the same classroom, this problem is equivalent to that of computing the maximum number of intervals that overlap at any point in time.

In [1]:
# True iff closed interval [start1,end1] overlaps [start2,end2].
function isoverlap((start1, end1), (start2, end2))
    # Overlap if start or end of one interval is within the other interval.
    start1 <= start2 && start2 <= end1 || start1 <= end2 && end2 <= end1 ||
        start2 <= start1 && start1 <= end2 || start2 <= end1 && end1 <= end2   
end 

isoverlap (generic function with 1 method)

In [2]:
using Test
a,b,c = (30, 75), (0, 50), (60, 150)
@test isoverlap(a, b)
@test isoverlap(a, c)
@test isoverlap(a, a)  # self-overlap
@test !isoverlap(b, c)
@test isoverlap((1,1),(0,50))

Test Passed

In [3]:
"True iff tuple (a, b) is a well-formed interval, i.e., with a <= b."
isinterval((a, b)) = a <= b

@test isinterval((0, 1))
@test isinterval((1, 1))
@test all(isinterval.([a, b, c]))

Test Passed

In [4]:
#=   INCORRECT:  This was an initial thought about the problem, but ignores overlaps of more than two.
function countoverlaps_BAD(intervals)
    maxoverlaps = 0
    for i in intervals
        overlaps = 0
        for j in intervals
            if isoverlap(i, j)
                println("$i and $j overlap")
                overlaps += 1
            end
        end
        if overlaps > maxoverlaps
            maxoverlaps = overlaps
        end
    end
    maxoverlaps
end     
=#

One brute force approach is to consider each moment of time (here an integer) between the first start and the last end time and count the number of intervals overlapping it, and then find the maximal such count.  This has complexity O(tn), where t is the number of time instants and n is the number of intervals.

In [5]:
function countoverlaps_brute(intervals)
    @assert all(isinterval.(intervals))
    mintime = minimum([i[1] for i in intervals])
    maxtime = maximum([i[2] for i in intervals])
    maxcount = 0
    for t in mintime:maxtime
        count = 0 
        for i in intervals
            if isoverlap((t,t), i)
                count += 1
            end
        end
        if count > maxcount
            maxcount = count
        end
    end
    maxcount
end

@test countoverlaps_brute([a,b,c]) == 2

Test Passed

We don't actually need to check every time instant, but just the first and last of each interval.

In [6]:
function countoverlaps_brute_ends(intervals)
    @assert all(isinterval.(intervals))
    maxcount = 0
    ts = vcat([i[1] for i in intervals],[i[2] for i in intervals])
    for t in ts
        count = 0  
        for i in intervals
            if isoverlap((t,t), i)
                count += 1
            end
        end
        if count > maxcount
            maxcount = count
        end
    end
    maxcount
end

@test countoverlaps_brute_ends([a,b,c]) == 2

Test Passed

We can also frame this as a maximal clique problem as follows.  We set up a graph where each interval is a node and a link exists between nodes i and j iff intervals i and j overlap.  A set {i1,...,ik} of intervals  overlapping at a time instant is equivalent to all pairs of intervals in the set overlapping, i.e., links between all pairs of nodes constituting the set, which means the set is a clique.  The size (cardinality) of the clique represents the number of classrooms required for those intervals, so the size of the maximum clique is the maximum number of classrooms ever required.

Consider a node i and a complete graph (which includes i) of size n (the number of nodes/intervals).  For each node j not linked to i, we reduce by 1 the count of the potential clique which include i.  (This ignores the absense of links between j and k, so isn't accurate, but is an upper bound.)  Find the minimum of these counts to bound the maximum clique size. 

In [7]:
function countoverlaps_clique_bound(intervals)
    notadj = [!isoverlap(i,j) for i in intervals, j in intervals]  # anti-adjacency matrix
    length(intervals) - maximum(sum(notadj, dims=1))
end

@test countoverlaps_clique_bound([a,b,c]) == 2

Test Passed